In [ ]:
#general imports. Add as I see fit.
import numpy as np
import pandas as pd

In [1]:
#General function for opening .txt files, which it seems like all the inputs are.
def get_input(filepath):
    f = open(filepath, "r")
    arr = f.read().split("\n")
    #NOTE: don't need the pop below if just copying the inputs, but will need it if saving directly from the webpage
    #arr.pop()
    return arr

In [ ]:
##NOTE - cells below that are not commented are works in progress/assorted trials.
##       Will work to comment in cells where it does work

In [ ]:
##SOLUTION CODE FOR 12/8 (part 1).

df = pd.read_csv('input_file_12_8.txt', sep=' ', header=None)
df.columns = ['Command', 'Iterator']

df['Visited'] = 0

def cycle_through(frame, accum, row=None):
    if not row:
        x = 0
    else:
        x = row
    if frame.loc[x, 'Visited'] == 1:
        return accum
    else:
        if frame.loc[x, 'Command'] == 'nop':
            frame.loc[x, 'Visited'] = 1
            return cycle_through(frame, accum, x+1)
        elif frame.loc[x, 'Command'] == 'acc':
            y = accum + frame.loc[x, 'Iterator']
            frame.loc[x, 'Visited'] = 1
            return cycle_through(frame, y, x+1)
        else:
            frame.loc[x, 'Visited'] = 1
            new_row = frame.loc[x, 'Iterator']
            return cycle_through(frame, accum, x + new_row)

#This will print correct answer for part 1, verified on website
print(cycle_through(df, 0))

df['Visited'] = 0

df['End Binary'] = 0
df.iloc[-1, 3] = 1

def replace(command):
    if command == 'nop':
        return 'jmp'
    elif command == 'jmp':
        return 'nop'
    else:
        return 'acc'
    
def cycle_through_new(frame, accum, row=None):
    if not row:
        x = 0
    else:
        x = row
        
    if frame.loc[x, 'End Binary'] == 1:
        return (accum, True)
    
    if frame.loc[x, 'Visited'] == 1:
        return (accum, False)
    else:
        #Copy here just to be safe
        new_frame = frame.copy()
        if frame.loc[x, 'Command'] == 'nop':
            new_frame.loc[x, 'Visited'] = 1
            return cycle_through_new(new_frame, accum, x+1)
        elif frame.loc[x, 'Command'] == 'acc':
            y = accum + frame.loc[x, 'Iterator']
            new_frame.loc[x, 'Visited'] = 1
            return cycle_through_new(new_frame, y, x+1)
        else:
            new_frame.loc[x, 'Visited'] = 1
            new_row = frame.loc[x, 'Iterator']
            return cycle_through_new(new_frame, accum, x + new_row)
        
for i in range(626):
    #Had to do this because simply assigning it to be the same would cause changes to propogate backwards
    #So annoying.
    df_new = df.copy()
    #print(df_new.iloc[i])
    df_new.iloc[i, 0] = replace(df_new.iloc[i, 0])
    #print(df_new.iloc[i])
    val = cycle_through_new(df_new, 0)
    if val[1] == True:
        print(val[0])
        break

In [ ]:
##SOLUTION CODE FOR 12_9 Part 1
#Gets solution for part 1
lst = get_input('input_file_12_9.txt')
lst = [int(i) for i in lst]

for i in range(26, len(lst)):
    temp_lst = lst.copy()[i-26:i-1]
    x = lst[i-1]
    break_bool = False
    #print(temp_lst)
    #print(x)
    for j in range(0, 24):
        for k in range(j+1, 25):
            if temp_lst[j] + temp_lst[k] == x:
                break_bool = True
            if break_bool:
                continue
        if break_bool:
            continue
    if not break_bool:
        print(x)         
print('Part 1 answer above')

##SOLUTION CODE FOR 12_9 Part 2
lst = get_input('input_file_12_9.txt')
lst = [int(i) for i in lst]

#Hard coded the solution for part 1. No need to not do it, the above code works.
solution = 90433990

def dec_9_function(lst, solution):
    for i in range(len(lst)):
        running_sum = 0
        counter = 0
        while running_sum < solution:
            running_sum = sum(lst[i:counter])
            if running_sum == solution:
                return max(lst[i:counter]) + min(lst[i:counter])
            counter += 1

print('Part 2 answer:')
print(dec_9_function(lst, solution))


In [60]:
##SOLUTION CODE FOR 12/10 Part 1
arr = get_input('input_file_12_10.txt')
arr = [int(i) for i in arr]
arr.sort()

def jolt_counter(lst):
    new_lst = []
    for i in range(len(lst)):
        try:
            new_lst.append(lst[i+1] - lst[i])
        #Accounts for the final thing, that isn't in the list but is guaranteed to be 3 higher
        except IndexError:
            new_lst.append(3)
    #Accounts for the first connection that also isn't in the list - will either be a 1 or 3.
    new_lst.append(lst[0])
    ones = len([i for i in new_lst if i == 1])
    twos = len([i for i in new_lst if i == 2])
    threes = len([i for i in new_lst if i == 3])
    
    return ones * threes

print(jolt_counter(arr))

##SOLUTION CODE FOR 12/10 PART 2
#NOTE - there are no gaps of 2 in the previous part.
#turns into a combinatorics problem with the 3 being buffers, the 1s being the possible combinations
def jolt_arrangements(lst):
    #Appending the start and ends on here
    new_lst = lst.copy()
    new_lst.append(0)
    new_lst.append(max(lst) + 3)
    new_lst.sort()
    lst_of_lsts = []
    lst_of_indices = []
    for i in range(len(lst)-1):
        try:
            if lst[i+1] - lst[i] == 3:
                lst_of_indices.append(i+1)
        except IndexError:
            pass
    #creating the new lists:
    lst_of_indices.insert(0, 0)
    lst_of_indices.append(len(new_lst))
    for i in range(len(lst_of_indices)):
        try:
            #NOTE the i+1 here removes 0 from the first list
            #That might be okay since that item is not replaceable.
            #Was not okay, added it back later. Still happy with the rest of the split, though, so leaving in
            lst_of_lsts.append(new_lst[lst_of_indices[i]+1:lst_of_indices[i+1]+1])
        except:
            pass
    #Now a combinatorics question based on the lists of lists - which items can be removed and it still work??
    #Hard coding adding the 0 to the first list, removing the end value from the last list
    lst_of_lsts[0].insert(0, 0)
    lst_of_lsts[-1].pop(-1)
    lst_to_multiply = []
    for i in lst_of_lsts:
        if len(i) == 1:
            continue
        #All of the below is for a more generalized solution, but ended up just hard coding patterns
        #None of the strings of 1s are longer than 3, so no hard math needed
        #Much more complicated if you do that.
        max_i = max(i)
        min_i = min(i)
        for j in i:
            if max_i - min_i == 3:
                dist_bool = True
            else:
                dist_bool = False
            min_needed = int((max_i - min_i) / 3)
        #Start of the hard coded possibilities, which is bad code but gets the job done.
        if len(i) == 3:
            lst_to_multiply.append(2)
        elif len(i) == 4:
            lst_to_multiply.append(4)
        elif len(i) == 5:
            lst_to_multiply.append(7)
    ##Can hard code possibilities here - the maximum length in list of lists is 5, which means 3 internal vals
    #Not good code, but gets the answer
    running_total = 1
    for i in lst_to_multiply:
        running_total = running_total * i
    return running_total
    

    
print(jolt_arrangements(arr))

2470
1973822685184


In [59]:
##SOLUTION CODE FOR 12/10 PART 2 (IN PROGRESS)
arr = get_input('input_file_12_10.txt')
arr = [int(i) for i in arr]
arr.sort()



([4, 7, 4, 4, 4, 2, 4, 7, 4, 2, 4, 2, 7, 7, 4, 4, 7, 2, 7, 4], [[0, 1, 2, 3], [6, 7, 8, 9, 10], [13, 14, 15, 16], [19], [22, 23, 24, 25], [28, 29], [32, 33], [36], [39], [42, 43, 44, 45], [48, 49, 50], [53], [56, 57, 58, 59], [62], [65], [68, 69, 70, 71, 72], [75], [78], [81, 82, 83, 84], [87], [90], [93, 94, 95], [98], [101, 102, 103, 104], [107, 108, 109], [112, 113, 114, 115, 116], [119, 120, 121, 122, 123], [126], [129, 130, 131, 132], [135], [138, 139, 140, 141], [144, 145], [148, 149, 150, 151, 152], [155, 156, 157], [160], [163], [166, 167, 168, 169, 170], [173, 174, 175, 176]], 1973822685184)


In [41]:
max(arr)

176

In [12]:
min(arr)

1